In [ ]:
import requests
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import datetime

# Fetch Earthquake Data from USGS API and filter for Canada
def fetch_earthquake_data(period='day'):
    url = f"https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_{period}.geojson"
    response = requests.get(url)
    data = response.json()
    
    features = data.get('features', [])
    records = []
    for quake in features:
        props = quake.get('properties', {})
        coords = quake.get('geometry', {}).get('coordinates', [])
        place = props.get('place', '')
        
        # Filter only earthquakes in Canada
        if "Canada" in place:
            records.append({
                'Magnitude': props.get('mag', 0),
                'Place': place,
                'Time': datetime.datetime.utcfromtimestamp(props.get('time', 0) / 1000),
                'Longitude': coords[0] if len(coords) > 0 else None,
                'Latitude': coords[1] if len(coords) > 1 else None,
                'Depth': coords[2] if len(coords) > 2 else None,
                'Region': place.split(',')[-1].strip() if ',' in place else 'Canada'
            })
    
    return pd.DataFrame(records)

# Initialize Dash App
app = dash.Dash(__name__)
server = app.server  # Required for deployment (e.g., Heroku, Render)
app.title = "Canada Earthquake Monitoring Dashboard"

# Layout
app.layout = html.Div([
    html.H1("Canada Earthquake Monitoring Dashboard", style={'textAlign': 'center'}),
    
    dcc.Interval(id='interval-component', interval=60000, n_intervals=0),
    
    dcc.Tabs([
        dcc.Tab(label='Real-Time Monitoring', children=[
            html.Div([dcc.Graph(id='live-map')]),
            html.Div([dash_table.DataTable(id='live-table', page_size=10)])
        ]),
        
        dcc.Tab(label='Statistical Analysis', children=[
            dcc.Dropdown(id='region-dropdown', multi=True, placeholder="Select Region"),
            dcc.Graph(id='magnitude-distribution'),
            dcc.Graph(id='depth-distribution')
        ])
    ])
])

# Callbacks
@app.callback(
    [Output('live-map', 'figure'), Output('live-table', 'data'), Output('region-dropdown', 'options')],
    Input('interval-component', 'n_intervals')
)
def update_real_time_data(n):
    df = fetch_earthquake_data('hour')  # Fetch hourly data for Canada
    if df.empty:
        return px.scatter_mapbox(title="No recent earthquakes in Canada"), [], []

    fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', color='Magnitude',
                             hover_name='Place', size='Magnitude',
                             mapbox_style='open-street-map', title='Real-Time Earthquake Monitoring (Canada)')
    
    table_data = df.to_dict('records')
    region_options = [{'label': region, 'value': region} for region in df['Region'].unique()]
    
    return fig, table_data, region_options

@app.callback(
    [Output('magnitude-distribution', 'figure'), Output('depth-distribution', 'figure')],
    Input('region-dropdown', 'value')
)
def update_statistical_analysis(selected_regions):
    df = fetch_earthquake_data('month')  # Fetch monthly data for Canada
    if df.empty:
        return px.histogram(title="No earthquake data available"), px.histogram(title="No earthquake data available")

    if selected_regions:
        df = df[df['Region'].isin(selected_regions)]
    
    mag_fig = px.histogram(df, x='Magnitude', nbins=20, title='Magnitude Distribution in Canada')
    depth_fig = px.histogram(df, x='Depth', nbins=20, title='Depth Distribution in Canada')
    
    return mag_fig, depth_fig

# Run Server
if __name__ == '__main__':
    app.run_server(debug=True)
